In [ ]:
import pandas as pd
import numpy as np

# 버전 확인
!pip list | findstr /r "pandas seaborn plotly"
pip install --upgrade seaborn

# 현재 폴더 확인 (Current Work Direction)
import os
os.getcwd()

# 현재 날짜 확인
from datetime import date as dt
dt.today()

# xls 파일 읽기
import openpyxl
df = pd.read_excel('./xxx.xlsx', sheet_name=2, header=None, names=['colname1','colname2'], na_values='-')

# 15줄 skip 후 읽음, cp949는 한글 인식, "./"는 현재 폴더
df = pd.read_csv('./xxx.csv', skiprows=15, encoding='CP949')

# 관측치 확인
df.tail(n=3)
df.head(5)
df.shape
df.index #행 확인
df.columns #col 확인
df.dtypes

# df 행열 선택해서 보기
df[0:5][['col1','col2','col3']]

# loc idx의 번호 그 자체 / iloc 순서상의 idx
df.loc[101:103, 'col']
df.iloc[0:3, [0,3,4]]

# rename() 활용 변수 이름 바꾸기 
df.rename(columns = {'col1':'var1', 'col2':'var2'})

# data type 변환 / flaot, object, category
df['col'].astype('float')

# ex) 53,542와 같이 object로 표현된 숫자, 숫자로 변형하기 
df['col'] = pd.to_numeric(df['col'].str.replace(',',''))

# filter( ) 메서드에서 변수 이름 패턴을 활용한 선택 
df.filter(regex='^s') # columns
    ## regex :  정규표현식(regular expression)
    ## '^s' : 's'로 시작하는 이름/텍스트
    ## 's' : 's'가 포함되는 이름/텍스트

# 수치형 변수만 선택 / include='number'/'object'...
df.select_dtypes(include='number')

# 한 단어 split하여 저장하기
df[['시','구','동']] = df['시군구'].str.split(' ',expand=True)

# df 합치기, ignore index하면 기존 index 무시 후 새로 setting, axis=0(아래), 1(오른쪽)
df = pd.concat([df1, df2, df3],ignore_index=True,axis=0) 

# folder 대상으로 한번에 불러오기
from glob import glob
dfs = [pd.read_csv(path_, skiprows=15, encoding='CP949') for path_ in glob('./*.csv')]
pd.concat(dfs).reset_index(drop=True)

# merge()를 활용한 결합 / inner left right outer / on은 기준 col
pd.merge(df_left, df_right, how='inner', on='col')

# idx 임의로 정하기
df['idx'] = list(range(101, 111))
df.set_index('idx', inplace=True)

# 변수 region의 수준 목록 확인 및 관심 수준 선택
df['col'].unique()
df['col'].nunique() # 개수

# drop()을 활용한 관측치/변수 제거(columns 활용)
df.drop(columns=['col'])
df.drop('col', axis=1)
    # axis = 0 : 관측치
    # axis = 1 : 변수

# 중복값 제거
df[['col1','col2']].drop_duplicates()

# isin()을 활용한 특정 수준 관측치 선택
df[(df['col1'] > 90) & (df['col2'].isin(['col2_value1','col2_value2']))]

# 글자 확인
df['col'].str.startswith('b')
df['col'].str.endswith('college')
df['col'].str.contains('degree')

# 범위 확인 / inclusive: 'both', 'neither', 'left', 'right'
df['col'].between(80, 90, inclusive='both')

# nlargest( ), nsmallest( )로 상위/하위 관측치 선택
df.sample(n=10)
df.nlargest(10, 'col')
df.nsmallest(10, 'col')

# quantile() / 0.25 Q1 Quartile / 0.5 Q2 median 
df['col'].quantile([0.0, 0.25, 0.5, 0.75, 1.0])

# 복수 기준의 설정 
df.sort_values(['col1', 'col2'], ascending=[True, False]).reset_index(drop=True)

# 집계값 계산 / mean, sum, var, std, count, size
df['col'].mean().round(2) # 소수점2자리
df['col'].value_counts() # col의 value cnt 값

# 그룹별 평균 계산(DataFrame 형식으로 출력)
df.groupby(by=['col1', 'col2'], as_index=False)['col3','col4'].mean()
df.groupby(by='col1', as_index=False)['col2'].agg(['min','max','mean'])
df.groupby(by='col1', as_index=False).agg({'col2':['min','max'],'col3':['mean']})

# cut()으로 구간화하기
df['new_col'] = pd.cut(df['col'], bins=10, labels=range(10)) # 등간격
df['new_col'] = pd.qcut(df['col'], q=10, labels=range(1,11)) # 등비율
pd.cut(df['col'], bins=[0,20,40,60,80,100]) #0~100 5구간
pd.cut(df['col'], bins=5) # 최소값부터 최대값까지 5구간

# 순위 생성
df['col'].rank(ascending=False) # 큰 수부터 # 동점일 경우 평균 등수
df['date'].rank(ascending=True, method='first') # 동점일 경우 index 순

# 그룹별 이동 값 변수 추가, id 기준 전날짜 구하기
df['date_prev'] = df.groupby('id')['date'].shift()
df['date_diff'] = df['date'] - df['date_prev']

# 그룹별 누적합 계산
df['cum_col'] = df.groupby('id')['col'].cumsum()

# 공분산 계산
df[['col1','col2']].cov()

# 상관계수 계산
df[['col1','col2']].corr()

# pivot_table()을 활용한 교차표 작성
    ## values : 값 변수
    ## index  : 행 그룹변수
    ## columns: 열 그룹변수 
    ## aggfunc: 집계 함수
    ## margin : 전체 평균값
pd.pivot_table(df, values='col1', index='col2', columns='col3', aggfunc='mean', margins=True)

# 하나라도 결측값이 있는 변수 확인
df.isnull() # 각 value 값 null 여부 확인
df.isnull().any(axis=0) # 각 열에 null 존재 확인

# 하나라도 결측값이 있는 관측치 확인
df[df.isnull().any(axis=1)]

# 하나라도 결측값이 있는 관측치 제거
df.dropna(subset=['col'])

# 모든 결측값을 일괄 대체
df.fillna(value=0)

# 변수별 결측값 대체 지정
df.fillna(value={'col1':0, 'col2':'NA'})

# 가장 앞쪽의 결측이 아닌 값으로 대체
    ## 센서 등의 값 누락에 활용
df.fillna(method='ffill')

# 이후 값중 결측이 아닌 값으로 대체
    ## groupby()를 활용하여 id기준 동일 값으로 대체
df.groupby('id').fillna(method='bfill')

# 특정한 변수만 결측값 대체
    ## groupby()와 fillna()를 활용할 경우 그룹변수가 사라짐
    ## 특정 변수만 선택해서 결측값 대체하고 업데이트
df['info2'] = df.groupby('col1')['col2'].fillna(method='ffill')  

# graph
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# 방법 1
df['col'].plot(kind='hist')
plt.show()

# 방법 2 / hist, boxplot, countplot
plt.hist(df['col'])

# 방법 3 boxplot, histplot, lmplot, lineplot
sns.histplot(data=df,x='x1',hue='legend')
sns.lmplot(data=df,x='x1',y='y1',hue='legend')
sns.heatmap(df, cmap='YlGnBu', annot=True, fmt='.2f')
# link 색참고 https://seaborn.pydata.org/tutorial/color_palettes.html

# plot 하나를 변수에 저장후 사진으로 저장 가능
plot1.figure.savefig('./plot1.jpg')

# 그래프 size 조절 방법
plt.figure(figsize=(3,3)) # 그릴때마다
plt.rcParams['figure.figsize'] = (4, 3)  # default로 지정

# 날짜인식으로 보기 이쁘게 나타내기
df['날짜'] = pd.to_datetime(df['날짜'])

# weekday / hour / day
df['월'] = df['날짜'].dt.month 

# 날짜 응용1
cond_month = df['월'].between(7,8)
cond_hour = df['시'].between(1,5)
df[cond_month & cond_hour].shape[0]

# 날짜 응용2
pt = df.pivot_table(values='col', index='월', columns='시', aggfunc='size')
pt.loc[7:8, 1:5].sum().sum()

##### 응용1) 그래프에 사각 표시하기 #####
from matplotlib.patches import Rectangle

#alpha는 투명의 정도 1은 불투명
sns.scatterplot(data=df,x='x1',y='y1',alpha=0.5) 
plt.axhline(mean_y) # 수평선 추가
plt.axvline(mean_x) # 수직선 추가

# 빨간색 상자 추가
rect = Rectangle((mean_x - 5, mean_y - 5), 10, 10, linewidth=1, color='red', alpha=0.3)
plt.gca().add_patch(rect)
plt.show()
######################################